In [ ]:
import torch
import numpy as np
from utils import *
from dataset import *
from noise2noise import Noise2Noise
from torch.utils.data import DataLoader

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

In [ ]:
# TODO
# validation batch_size, 
# true_image dataset extraction loop
# image_summary [index] : noise2noise.py : summary.py
config = {
    'device':device,
    'dataset':'gaussian',
    'batch_size_train':12,
    'batch_size_valid':1,
    'num_workers':5,
    'learning_rate':0.001,
    'checkpoint_dir': './models_trained/.',
    'checkpoint_filename': 'latest.pt',
    'root_dir':'.',
    'log_dir': './log/',
    'data_folder':'./data/',
    'num_epochs': 100,
}


In [ ]:
trainloader = datasetLoader(config, 'train')
validloader = datasetLoader(config, 'valid')

In [ ]:
NET = Noise2Noise(config, trainloader, validloader,trainable=True)

In [ ]:
NET.load_model(config)

In [ ]:
NET.loss_size

In [ ]:
NET.train(config)

In [ ]:
NET.epoch

In [ ]:
load()

In [ ]:
true_images = trueImages.load_data()
valid_images = validloader.load_data()

In [ ]:
from logger import Logger
from summary import viewSummary
summary = viewSummary(config['log_dir'])
import time

In [ ]:
for epoch,(t,v) in enumerate(zip(true_images, valid_images)):
    
    src, trgt = v
    summary.view_image({'true_images':src.view(-1,3,256,256)[:2].cpu().numpy(),
                                     'output_images':t.view(-1,3,256,256)[:2].detach().cpu().numpy()}, epoch)
    time.sleep(15)

In [ ]:
def true_image_dataset(config, image_name):
    import os
    from skimage import io
    import torchvision.transforms as transforms
    
    path = os.path.join(config['root_dir'], config['data_folder'])
    image_path = path + image_name
    image = io.imread(image_path)
    try:
        if len(np.shape(image))==2:
            image = np.stack((image)*3, axis=-1)
    
        transform_list = [transforms.ToPILImage(),
                          transforms.Resize((256,256)),
                          transforms.ToTensor()]

        transform = transforms.Compose(transform_list)
        img = transform(image)
        return img
    except:
        print(image_path, image.shape)

In [ ]:
img = true_image_dataset(config, 'ILSVRC2012_val_00028627.JPEG')

In [ ]:
import matplotlib

In [ ]:
from matplotlib import pyplot

In [ ]:
pyplot.imshow(img)

In [ ]:
img.view(-1,3,256,256)[:1]